In [10]:
import os
import torch
import numpy as np
import sys
sys.path.append('../')

from models.mcnnpytorch.src.crowd_count import CrowdCounter
from models.mcnnpytorch.src import network
from models.mcnnpytorch.src.data_loader import ImageDataLoader
from models.mcnnpytorch.src.timer import Timer
from models.mcnnpytorch.src import utils
from models.mcnnpytorch.src.evaluate_model import evaluate_model

In [11]:
method = 'mcnn'
dataset_name = 'shtechA'
output_dir = './saved_models/'

train_path = './data/SHT/part_A_final/train_data/images'
train_gt_path = './data/SHT/part_A_final/train_data/ground_truth'
val_path = './data/SHT/part_A_final/test_data/images'
val_gt_path = './data/SHT/part_A_final/test_data/ground_truth'